입력은 user_id, 출력은 딕셔너리, 혹은 데이터프레임으로 가공 가능

In [ ]:
#서프라이즈 패키지 설치-필수
!pip install scikit-surprise

In [ ]:
#피클파일 불러오기 예시-필수
import pickle

with open("algo.pickle","rb") as fr:
  algo = pickle.load(fr) #반드시 algo변수명으로 선언

In [ ]:
# csv파일 불러오기-필수
import pandas as pd
from ast import literal_eval
products = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/products.csv') #경로는 파일경로에 맞게, 반드시 products변수명으로 선언
up_reorder = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/CP1/up_reorder2.csv') #경로는 파일경로에 맞게, 반드시 up_reorder변수명으로 선언
up_reorder['product_id'] = up_reorder['product_id'].apply(literal_eval)

print('products shape :', products.shape)
print('up_reorder shape :', up_reorder.shape)

In [ ]:
# 보조함수들 선언-필수
def sortkey_est(pred):
  return pred.est

def find_name(arg):
  return products[products['product_id']==arg]['product_name'].values[0]

In [ ]:
#1. 소분류/대분류로 테이블 만들어주는 형식-선택
def recommend(user_id):
  dic = {}
  try:
    a = up_reorder[up_reorder['user_id']==user_id]['product_id'].values[0]
  except:
    return 'No recommend'
  for i in a:
    aisle = products[products['product_id']==i]['aisle_id'].values[0]
    department = products[products['product_id']==i]['department_id'].values[0]
    candi = products[(products['aisle_id']==aisle) & (products['department_id']==department)]['product_id'].to_list()
    candi.remove(i)
    predi = [algo.predict(str(user_id), str(j)) for j in candi]
    predi.sort(key=sortkey_est, reverse=True)
    top_predi = predi[:3]
    top_product_id = [int(pred.iid) for pred in top_predi]
    top_product_name = products[products.product_id.isin(top_product_id)]['product_name'].to_list()
    dic[find_name(aisle)+'/'+find_name(department)] = top_product_name
  return dic

In [ ]:
#유저아이디 1번 추천리스트 불러오기, 결과는 딕셔너리
recommend(1)

In [ ]:
#데이터 프레임으로도 살펴보기
pd.DataFrame(recommend(1))

원하시는 형식으로 사용하시면 될 것 같습니다.

In [ ]:
# 2. 그냥 추천상품 열거하는 형식(이번엔 top2가 아니라 top3까지만=>리스트 길이 때문에)-선택
def recommend2(user_id):
  dic2 = []
  try:
    a = up_reorder[up_reorder['user_id']==user_id]['product_id'].values[0]
  except:
    return 'No recommend'
  for i in a:
    aisle = products[products['product_id']==i]['aisle_id'].values[0]
    department = products[products['product_id']==i]['department_id'].values[0]
    candi = products[(products['aisle_id']==aisle) & (products['department_id']==department)]['product_id'].to_list()
    candi.remove(i)
    predi = [algo.predict(str(user_id), str(j)) for j in candi]
    predi.sort(key=sortkey_est, reverse=True)
    top_predi = predi[:2]
    top_product_id = [int(pred.iid) for pred in top_predi]
    top_product_name = products[products.product_id.isin(top_product_id)]['product_name'].to_list()
    dic2.extend(top_product_name)
  return dic2

In [ ]:
#예시, 결과는 리스트
recommend2(1)